In [1]:
!pip install langchain langchain_chroma langchain_core langchain_openai langchain_community pypdf

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Bibliotecas para Cadeias de Documentos e Recuperação
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

# Biblioteca para Armazenamento e Recuperação
from langchain_chroma import Chroma

# Bibliotecas para Modelos e Prompts
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

# Biblioteca para Divisão de Texto
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Bibliotecas para Embeddings e Modelos de Linguagem
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

# Biblioteca para Carregamento de Documentos
from langchain_community.document_loaders import PyPDFLoader


In [3]:
# Caso não seja possivel colocar a chave nas variaveis de ambiente insira manualmente aqui
openai_api_key="sk-proj-NNyucbht5JL4bDirqYaHT3BlbkFJg1VkFu5D1J7rHJobux4q"

# Instânciando o modelo
llm = ChatOpenAI(
    model = "gpt-4o",
    # A temperatura varia de 0 ate 1, aleatoriedade da resposta
    temperature = 0.7, 
    max_tokens = None,
    api_key = openai_api_key,
    # timeout=None,
    # max_retries=2
)

In [4]:
# 1. Carregar dividir e indexar o conteudo do arquivo
print("Inicializando")

# Carrega o arquivo do PDF
loader = PyPDFLoader("/home/claudio_augusto/WEB_TECH/assistente-carreiras/assistente-carreiras-gpt/assets/DATA/Cursos_completos.pdf", extract_images=False)
# loader = PyPDFLoader("assets/DATA/Cursos_completos.pdf", extract_images=False)
docs = loader.load()

"""
    Args:
        chunck_size (int): tamanho das partes do texto
            Impacto: Tamanhos menores podem levar a uma maior granularidade, facilitando o modelo a captar detalhes específicos, 
        mas podem também introduzir mais descontinuidade e perder contexto. 
                     Tamanhos maiores mantêm mais contexto, mas podem dificultar o processamento devido ao limite de tokens do modelo.

        chunck_overlap (int): tamanho da sobreposição entre as partes consecutivas do texto.
            Impacto: Uma maior sobreposição ajuda a manter o contexto entre os chunks, mas aumenta o volume de dados a serem processados. 
                     Menor sobreposição pode acelerar o processamento, mas corre o risco de perder conexões importantes entre os chunks.

    Atenção: tradeoff entre eficiência de processamento e compreensão do contexto. Testar diversos parâmetros.
"""

# Inicializa o objeto de divisão de texto
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# Divide o texto em partes menores
splits = text_splitter.split_documents(docs)

# Cria uma representação vetorial dos documentos para facilitar a busca semântica
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=openai_api_key))
# Cria um recuperador de documentos baseado no vetorstore
retriever = vectorstore.as_retriever()

Inicializando


In [5]:
# 2. Incorporar o retriver dentro da corrente de resposta do chat ------------------------------------------------------

system_prompt = (
    "Você é um assistente que auxilia pessoas a acharem o curso ideal para elas"
    "O dados cursos estao contidos no pdf que lhe foi fornecido"
    "Você deve utilizar como base apenas os cursos do pdf"
    "E não deve responder nada alem do que uma assistente de carreiras saberia"
    "Restrinja-se apenas ao documento, não tire nada por fora"
    "Você esta a serviso da universidade PUC minas (Pontificia Univercidade Catolica)"
    "\n\n"
    "{context}"
)

# system_prompt = (
    # "Você é um assistente de carreiras especializado nos cursos de pós-graduação lato sensu da PUC Minas."
    # "Seu trabalho é auxiliar candidatos na escolha do curso de tecnologia mais adequado à suas carreiras, interesses e perfis. Para isso, siga as seguintes regras:"
    # "1) Fornecer informações baseadas nos dados oficiais da PUC Minas. Evite fornecer informações externas."
    # "2) Obter dados base do usuário através de conversa, como formação, experiência profissional e interesses."
    # "2) Considerar o histórico do candidato, como formação, experiência profissional e interesses, para oferecer sugestões personalizadas."
    # "3) Utilizar perguntas abertas e linguagem natural para criar uma interação amigável e engajadora."
    # "4) Manter o foco em cursos de tecnologia da PUC Minas, evitando informações externas ou sugestões de outras instituições. Caso ocorra algo fora do escopo de orientação de cursos, retorne: 'Sou um assistente de carreiras. Posso te ajudar a escolher o melhor curso baseado em seu perfil, esclarecer dúvidas sobre os cursos e comparar cursos do seu interesse.'"
    # "5) Forneça mais de um curso ao candidato, para que ele possa compará-los. Evite fornecer apenas um."
    # "6) Forneça as informações em tópicos, com a seguinte estrutura:"
    # "- [NOME DO CURSO:]"
    # "- [OBJETIVOS:]"
    # "- [JUSTIFICATIVA:]"
    # )

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Adicionando Historico do chat
# TODO: Checar, tá com o mesmo comando do System.prompt.
contextualize_q_system_prompt = (
    "O aluno ira te repassar perguntas relacionadas aos cursos"
    "A duvida pode ser relacionada a uma diciplina a um curso ou alguma ajuda mara escolher um curso"
    "VocÊ apenas tem acesso a cursos da pos-gradução PUC Minas"
    "VOCE NÃO DEVE RESPONDER NADA QUE NÃO ESTEJA RELACIONADO A CURSOS"
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

#Corrente 
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# Vetor do chat
chat_history = []

while True:
  # Pergunta ao usuário
  question = input("Digite sua pergunta: ")

  # Processa a pergunta e gera a resposta da IA
  ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
  chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1["answer"])])

  # Imprime a resposta da IA
  print(ai_msg_1["answer"] + "\n")

In [ ]:

# Cria uma representação vetorial dos documentos para facilitar a busca semântica
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=openai_api_key))

# Cria um recuperador de documentos baseado no vetorstore
retriever = vectorstore.as_retriever()

# 2. Incorporar o retriever dentro da corrente de resposta do chat

# Definição do prompt do sistema, que orienta a IA sobre seu papel e restrições
system_prompt = (
    "Você é um assistente que auxilia pessoas a acharem o curso ideal para elas. "
    "Os dados dos cursos estão contidos no PDF que lhe foi fornecido. "
    "Você deve utilizar como base APENAS OS CURSOS DO PDF "
    "E não deve responder nada além do que um assistente de carreiras saberia. "
    "RESTRINJA-SE APENAS AO QUE ESTÁ NO DOCUMENTO, NÃO TIRE NADA POR FORA. "
    "Você está a serviço da universidade PUC Minas (Pontifícia Universidade Católica)."
    "\n\n"
    "{context}"
)

# Cria o template de prompt para o chat, que inclui o prompt do sistema e a entrada do usuário
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Cria a cadeia de perguntas e respostas usando os documentos carregados
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Cria a cadeia de recuperação que utiliza o retriever e a cadeia de perguntas e respostas
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Adicionando Histórico do chat
# Prompt que contextualiza o sistema sobre a natureza das perguntas que ele receberá
contextualize_q_system_prompt = (
    "O aluno irá te repassar perguntas relacionadas aos cursos. "
    "A dúvida pode ser relacionada a uma disciplina, a um curso ou alguma ajuda para escolher um curso. "
    "Você apenas tem acesso a cursos da pós-graduação PUC Minas. "
    "VOCÊ NÃO DEVE RESPONDER NADA QUE NÃO ESTEJA RELACIONADO A CURSOS."
)

# Cria o template de prompt para o chat que leva em consideração o histórico do chat
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Cria um recuperador que leva em conta o histórico do chat
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Template de prompt para o chat que leva em consideração o histórico do chat
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Corrente de perguntas e respostas usando os documentos e considerando o histórico
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Cria a cadeia de recuperação que utiliza o recuperador com histórico e a cadeia de perguntas e respostas
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

# Vetor do histórico do chat
chat_history = []

while True:
    # Pergunta ao usuário
    question = input("Digite sua pergunta: ")

    # Processa a pergunta e gera a resposta da IA
    ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
    
    # Adiciona a pergunta do usuário e a resposta da IA ao histórico do chat
    chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1["answer"])])

    # Imprime a resposta da IA
    print(ai_msg_1["answer"] + "\n")

### Explicações Adicionais:
- **PyPDFLoader:** Carrega o conteúdo do PDF.
- **RecursiveCharacterTextSplitter:** Divide o texto do PDF em pedaços menores para processamento.
- **Chroma:** Utiliza embeddings (representações vetoriais) para criar um índice de busca semântica.
- **ChatPromptTemplate:** Define como a conversa será estruturada, incluindo as mensagens do sistema e do usuário.
- **create_stuff_documents_chain e create_retrieval_chain:** Funções que criam as cadeias de processamento para recuperação e resposta baseadas nos documentos carregados.
- **chat_history:** Mantém o histórico das interações para contexto adicional nas respostas futuras.